[![deep-learning-notes](https://github.com/semilleroCV/deep-learning-notes/raw/main/assets/banner-notebook.png)](https://github.com/semilleroCV/deep-learning-notes)

## MobileNet V1

In [3]:
%%capture
#@title **Install required packages**

! pip install torchinfo

In [5]:
#@title **Importing libraries**

import torch # 2.3.1+cu121
import torchinfo #1.8.0

import torch.nn as nn
from torch import Tensor

In [6]:
# Note: Not all dependencies have the __version__ method.

print(f"torch version: {torch.__version__}")
print(f"torchinfo version: {torchinfo.__version__}")

torch version: 2.3.1+cu121
torchinfo version: 1.8.0


#### Mobilenet V1 architecture code

In [12]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int):
        super(ConvBlock, self).__init__()

        self.conv_blk = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.conv_blk(x)


class DepthwiseConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int):
        super(DepthwiseConvBlock, self).__init__()

        self.depthwise_conv_blk = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.depthwise_conv_blk(x)


class MobileNetV1(nn.Module):
    def __init__(self, layer_config: list, depth_multiplier: int, num_classes: int = 1000):
        super(MobileNetV1, self).__init__()

        """depth multiplier is also called width_multiplier or alpha"""

        self.model = nn.Sequential()

        self.model.add_module('conv_blk_1', ConvBlock(3, 32, 2))

        for idx, params in enumerate(layer_config):
          """layer_params: List -> (in_channels, out_channels, stride)"""
          self.model.add_module(f"dw_blk_{idx}",DepthwiseConvBlock(int(params[0]*depth_multiplier),
                                                                   (params[1]*depth_multiplier), params[2]))
          
        self.model.add_module('pool', nn.AdaptiveAvgPool2d(1))
        self.model.add_module('flatten', nn.Flatten())
        self.model.add_module('fc',nn.Linear(1024, num_classes))

    def forward(self, x):
        return self.model(x)

In [14]:
# Declare parameters for depth-wise separable convolution layers -> (in_channels, out_channels, stride)
dw_params = [
    (32, 64, 1),
    (64, 128, 2),
    (128, 128, 1),
    (128, 256, 2),
    (256, 256, 1),
    (256, 512, 2),
    (512, 512, 1),
    (512, 512, 1),
    (512, 512, 1),
    (512, 512, 1),
    (512, 512, 1),
    (512, 1024, 2),
    (1024, 1024, 1),
]

model = MobileNetV1(layer_config=dw_params, depth_multiplier=1, num_classes=1000)
torchinfo.summary(model, (3, 224, 224), batch_dim = 0)

Layer (type:depth-idx)                   Output Shape              Param #
MobileNetV1                              [1, 1000]                 --
├─Sequential: 1-1                        [1, 1000]                 --
│    └─ConvBlock: 2-1                    [1, 32, 112, 112]         --
│    │    └─Sequential: 3-1              [1, 32, 112, 112]         928
│    └─DepthwiseConvBlock: 2-2           [1, 64, 112, 112]         --
│    │    └─Sequential: 3-2              [1, 64, 112, 112]         2,528
│    └─DepthwiseConvBlock: 2-3           [1, 128, 56, 56]          --
│    │    └─Sequential: 3-3              [1, 128, 56, 56]          9,152
│    └─DepthwiseConvBlock: 2-4           [1, 128, 56, 56]          --
│    │    └─Sequential: 3-4              [1, 128, 56, 56]          18,048
│    └─DepthwiseConvBlock: 2-5           [1, 256, 28, 28]          --
│    │    └─Sequential: 3-5              [1, 256, 28, 28]          34,688
│    └─DepthwiseConvBlock: 2-6           [1, 256, 28, 28]          --
